#### Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import concurrent.futures
from datetime import datetime, timedelta
import json
import tmdbsimple as tmdb
import threading

from multiprocessing import Pool
from requests.exceptions import RequestException

from langdetect import detect

-----
-----
-----

### till 1900

In [2]:
start_time = time.time()

results = []
start_date = "1800-01-01"
end_date = "1900-01-01"
base_url = "https://api.themoviedb.org/3/discover/movie"
headers = {
    "accept": "application/json",
    "Authorization": "Bearer YOUR_BEARER" ######### ------------------------- FROM TMDB API
}

def fetch_data(page):
    params = {
        "include_adult": "false",
        "include_video": "false",
        "language": "en-US",
        "page": str(page),
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "sort_by": "primary_release_date.asc"
    }
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get("results", [])
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching data for page {page}: {e}")
        return []

# Fetch the first page to get the total number of pages
first_page_url = base_url + "?include_adult=false&include_video=false&language=en-US&page=1&primary_release_date.gte=" + start_date + "&primary_release_date.lte=" + end_date + "&sort_by=primary_release_date.asc"

try:
    first_page_response = requests.get(first_page_url, headers=headers)
    first_page_response.raise_for_status()
    first_page_data = first_page_response.json()
    pages = first_page_data.get("total_pages", 0)
except requests.exceptions.RequestException as e:
    print(f"Error occurred while fetching the first page: {e}")
    pages = 0

# Use ThreadPoolExecutor for concurrent processing
with ThreadPoolExecutor() as executor:
    # Fetch data for multiple pages concurrently
    page_numbers = range(1, pages)
    data_list = executor.map(fetch_data, page_numbers)

    # Process the retrieved data
    for data in data_list:
        results.extend(data)

# Convert the results to a DataFrame
results= pd.DataFrame(results)

# Calculate the total execution time
total_time = time.time() - start_time

# Print the total execution time
print(f"Total execution time: {total_time} seconds")

results

Total execution time: 2.7354648113250732 seconds


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/6vYffq6NoneNmNPp6IsobnnlrHS.jpg,[99],315946,fr,Passage de Venus,Photo sequence of the rare transit of Venus ov...,5.240,/XWPDZzK7N2WQcejI8W96IxZEeP.jpg,1874-12-09,Passage of Venus,False,6.2,89
1,False,None,[16],1146511,en,Les Scieurs de Long,Praxinoscope animation of two workers sawing w...,0.600,/rspaui7S34F0UWzM7d1eOd9V9BR.jpg,1878-05-07,Les Scieurs de Long,False,2.0,1
2,False,None,[16],1146099,en,Le Moulin à Eau,Praxinoscope animation of the water mill. Seri...,0.600,/kcn9YoDTSvj4YORVS9ZKdQ6WcJh.jpg,1878-05-07,Le Moulin à Eau,False,4.0,1
3,False,None,[16],1144949,en,Les Petits Valseurs,Praxinoscope animation of kids dancing. Series...,0.600,/nSjNLZvkHtmsEDMvNwmejZRfvyF.jpg,1878-05-07,Les Petits Valseurs,False,4.0,1
4,False,None,[16],1144947,en,Les Clowns,Praxinoscope animation of clowns dancing with ...,0.600,/18hsZcbtoJmMWh5Q0hi66oyQ2xs.jpg,1878-05-07,Les Clowns,False,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,False,None,[35],106815,fr,Le réveil d'un monsieur pressé,A man misses his train due to his clothes turn...,1.291,/1tiLrYzpktFOTLc0XDAcwIz716R.jpg,1900-01-01,How He Missed His Train,False,5.6,22
2036,False,None,[35],106814,fr,Le Savant et le Chimpanzé,A monkey wreaks havoc on a doctor.,0.849,/8rBpNnQbMctVgUOxyZEMP1T2fls.jpg,1900-01-01,The Doctor and the Monkey,False,5.4,24
2037,False,None,[],106812,fr,Le Tonneau des Danaïdes,A magician stuffs eight of his lovely assistan...,0.960,/xaD2Bcz85ACn8qZvcMsAMkDmvkp.jpg,1900-01-01,Eight Girls in a Barrel,False,5.2,19
2038,False,None,[35],106809,fr,Le déshabillage impossible,A man takes off his clothes in preparation for...,1.427,/5OOH1exeqx6IfLGL3UcQsGVJa6a.jpg,1900-01-01,Going to Bed Under Difficulties,False,6.5,39


### till the 00's

In [3]:
start_time = time.time()

base_url = "https://api.themoviedb.org/3/discover/movie"
headers = {
    "accept": "application/json",
    "Authorization": "Bearer YOUR_BEARER" ######### ------------------------- FROM TMDB API
}

def fetch_data(year):
    start_date = str(year) + "-01-02"
    end_date = str(year + 1) + "-01-01"
    params = {
        "include_adult": "false",
        "include_video": "false",
        "language": "en-US",
        "page": "1",
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "sort_by": "primary_release_date.asc"
    }
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        pages = data.get("total_pages", 0)
        result_df = pd.DataFrame(data.get("results", []))
        for page in range(2, pages):
            params["page"] = str(page)
            response = requests.get(base_url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            result_df = pd.concat([result_df, pd.DataFrame(data.get("results", []))], ignore_index=True)
        return result_df
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching data for year {year}: {e}")

# Fetch data for the years before 1900
results = results

with concurrent.futures.ThreadPoolExecutor() as executor:
    years = range(1900,2000)
    future_to_year = {executor.submit(fetch_data, year): year for year in years}

    for future in concurrent.futures.as_completed(future_to_year):
        year = future_to_year[future]
        try:
            result_df = future.result()
            results = pd.concat([results, result_df], ignore_index=True)
        except Exception as e:
            print(f"Error occurred for year {year}: {e}")

# Calculate the total execution time
total_time = time.time() - start_time

# Print the total execution time
print(f"Total execution time: {total_time} seconds")

results

Total execution time: 316.1715352535248 seconds


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/6vYffq6NoneNmNPp6IsobnnlrHS.jpg,[99],315946,fr,Passage de Venus,Photo sequence of the rare transit of Venus ov...,5.240,/XWPDZzK7N2WQcejI8W96IxZEeP.jpg,1874-12-09,Passage of Venus,False,6.2,89
1,False,None,[16],1146511,en,Les Scieurs de Long,Praxinoscope animation of two workers sawing w...,0.600,/rspaui7S34F0UWzM7d1eOd9V9BR.jpg,1878-05-07,Les Scieurs de Long,False,2.0,1
2,False,None,[16],1146099,en,Le Moulin à Eau,Praxinoscope animation of the water mill. Seri...,0.600,/kcn9YoDTSvj4YORVS9ZKdQ6WcJh.jpg,1878-05-07,Le Moulin à Eau,False,4.0,1
3,False,None,[16],1144949,en,Les Petits Valseurs,Praxinoscope animation of kids dancing. Series...,0.600,/nSjNLZvkHtmsEDMvNwmejZRfvyF.jpg,1878-05-07,Les Petits Valseurs,False,4.0,1
4,False,None,[16],1144947,en,Les Clowns,Praxinoscope animation of clowns dancing with ...,0.600,/18hsZcbtoJmMWh5Q0hi66oyQ2xs.jpg,1878-05-07,Les Clowns,False,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270135,False,None,"[53, 18]",28085,en,Blind Target,"A popular American novelist, who has written h...",1.261,/wRsJ9YLOJKtdaXZbO6hEBsyp9tU.jpg,2000-01-01,Blind Target,False,3.3,7
270136,False,None,[27],27879,en,Zombie Bloodbath 3: Zombie Armageddon,A bunch of zombies that are used by the govern...,1.257,/qb1MPIfHu7ZWVA8Z193w4pFdZP.jpg,2000-01-01,Zombie Bloodbath 3: Zombie Armageddon,False,6.3,3
270137,False,/7TbNMrlH90RBOTpUwQvMOvKaVD1.jpg,[18],20152,is,Englar alheimsins,The story of an Icelandic man and his slow des...,1.585,/8RViMuKJVmRamRLaO1wl3a5a3EL.jpg,2000-01-01,Angels of the Universe,False,6.7,27
270138,False,None,[],18368,en,Nursery Rhymes,,0.600,None,2000-01-01,Nursery Rhymes,False,0.0,0


In [4]:
results_bef_2000=results

## from 2000 forward
---

In [5]:
###### Movie importation time: +- 15min.

def fetch_data(start_date, end_date):
    url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=1&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&sort_by=primary_release_date.asc"
    headers = {
        "accept": "application/json",
        "Authorization": "Bearer YOUR_BEARER" ######### ------------------------- FROM TMDB API
    }

    page = 1
    total_pages = float("inf")
    results = pd.DataFrame()

    while page <= total_pages:
        page_url = f"{url}&page={page}"
        response = requests.get(page_url, headers=headers)
        data = response.json()
       # print(json.dumps(data, indent=4))
        new_data = pd.DataFrame(data.get("results", []))
        results = pd.concat([results, new_data], ignore_index=True)
        total_pages = data.get("total_pages", 0)
        page += 1
    
    #print(f"Fetched data for {start_date} - {end_date}")
    return results

def process_batch(start_date, end_date):
    results = pd.DataFrame()
    with ThreadPoolExecutor() as executor:
        futures = []
        current_date = start_date
        while current_date <= end_date:
            next_date = current_date + timedelta(days=30)
            if next_date > end_date:
                next_date = end_date

            futures.append(executor.submit(fetch_data, current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))
            current_date = next_date + timedelta(days=1)

        for future in futures:
            result_df = future.result()
            results = pd.concat([results, result_df], ignore_index=True)

    return results

start_date = datetime(2000, 1, 2)

#getting the newest date

url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=1&sort_by=primary_release_date.desc"
headers = {
    "accept": "application/json",
    "Authorization": "Bearer YOUR_BEARER" ######### ------------------------- FROM TMDB API
}

response = requests.get(url, headers=headers)
data = response.json()
last_date = data["results"][0]["release_date"]
last_date = datetime.strptime(last_date, "%Y-%m-%d")

start_time = time.time()
results = process_batch(start_date, last_date)
total_time = time.time() - start_time

results = pd.concat([results_bef_2000, results], ignore_index=True)

print(f"Total execution time: {total_time} seconds")

results

Total execution time: 521.8900218009949 seconds


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/6vYffq6NoneNmNPp6IsobnnlrHS.jpg,[99],315946,fr,Passage de Venus,Photo sequence of the rare transit of Venus ov...,5.240,/XWPDZzK7N2WQcejI8W96IxZEeP.jpg,1874-12-09,Passage of Venus,False,6.2,89
1,False,None,[16],1146511,en,Les Scieurs de Long,Praxinoscope animation of two workers sawing w...,0.600,/rspaui7S34F0UWzM7d1eOd9V9BR.jpg,1878-05-07,Les Scieurs de Long,False,2.0,1
2,False,None,[16],1146099,en,Le Moulin à Eau,Praxinoscope animation of the water mill. Seri...,0.600,/kcn9YoDTSvj4YORVS9ZKdQ6WcJh.jpg,1878-05-07,Le Moulin à Eau,False,4.0,1
3,False,None,[16],1144949,en,Les Petits Valseurs,Praxinoscope animation of kids dancing. Series...,0.600,/nSjNLZvkHtmsEDMvNwmejZRfvyF.jpg,1878-05-07,Les Petits Valseurs,False,4.0,1
4,False,None,[16],1144947,en,Les Clowns,Praxinoscope animation of clowns dancing with ...,0.600,/18hsZcbtoJmMWh5Q0hi66oyQ2xs.jpg,1878-05-07,Les Clowns,False,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711873,False,None,[27],1127894,en,Birdemic 5: Birdman,Plot details are currently under wraps but exp...,1.153,None,2027-01-01,Birdemic 5: Birdman,False,0.0,0
711874,False,/jqFC2WjYF07hx2X7cs0XmY9jBX6.jpg,"[28, 878]",1003598,en,Avengers: Secret Wars,An upcoming film in the Marvel Cinematic Unive...,23.273,/8chwENebfUEJzZ7sMUA0nOgiCKk.jpg,2027-05-05,Avengers: Secret Wars,False,0.0,0
711875,False,None,"[28, 878, 12, 14, 53]",216527,en,Avatar 4,,19.103,/qzMYKnT4MG1d0gnhwytr4cKhUvS.jpg,2029-12-19,Avatar 4,False,0.0,0
711876,False,None,"[28, 12, 878]",393209,en,Avatar 5,,15.253,/rtmmvqkIC5zDMEd638Es2woxbz8.jpg,2031-12-17,Avatar 5,False,0.0,0


In [6]:
df=results

## Genres

In [7]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer YOUR_BEARER" ######### ------------------------- FROM TMDB API
}

response = requests.get(url, headers=headers)

genres_dict = json.loads(response.text)

# Extract the genres list from the dictionary
genres_list = genres_dict['genres']

# Create a DataFrame from the genres list
genres = pd.DataFrame(genres_list)
genres

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [8]:
df['genre_ids'] = df['genre_ids'].astype(str)

df['genre_ids'] = df['genre_ids'].str.replace("[", "")
df['genre_ids'] = df['genre_ids'].str.replace("]", "")
df['genre_ids'] = df['genre_ids'].str.replace("'", "")

C:\Users\josep\AppData\Local\Temp\ipykernel_21140\990186944.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['genre_ids'] = df['genre_ids'].str.replace("[", "")
C:\Users\josep\AppData\Local\Temp\ipykernel_21140\990186944.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['genre_ids'] = df['genre_ids'].str.replace("]", "")


In [9]:
df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/6vYffq6NoneNmNPp6IsobnnlrHS.jpg,99,315946,fr,Passage de Venus,Photo sequence of the rare transit of Venus ov...,5.240,/XWPDZzK7N2WQcejI8W96IxZEeP.jpg,1874-12-09,Passage of Venus,False,6.2,89
1,False,None,16,1146511,en,Les Scieurs de Long,Praxinoscope animation of two workers sawing w...,0.600,/rspaui7S34F0UWzM7d1eOd9V9BR.jpg,1878-05-07,Les Scieurs de Long,False,2.0,1
2,False,None,16,1146099,en,Le Moulin à Eau,Praxinoscope animation of the water mill. Seri...,0.600,/kcn9YoDTSvj4YORVS9ZKdQ6WcJh.jpg,1878-05-07,Le Moulin à Eau,False,4.0,1
3,False,None,16,1144949,en,Les Petits Valseurs,Praxinoscope animation of kids dancing. Series...,0.600,/nSjNLZvkHtmsEDMvNwmejZRfvyF.jpg,1878-05-07,Les Petits Valseurs,False,4.0,1
4,False,None,16,1144947,en,Les Clowns,Praxinoscope animation of clowns dancing with ...,0.600,/18hsZcbtoJmMWh5Q0hi66oyQ2xs.jpg,1878-05-07,Les Clowns,False,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711873,False,None,27,1127894,en,Birdemic 5: Birdman,Plot details are currently under wraps but exp...,1.153,None,2027-01-01,Birdemic 5: Birdman,False,0.0,0
711874,False,/jqFC2WjYF07hx2X7cs0XmY9jBX6.jpg,"28, 878",1003598,en,Avengers: Secret Wars,An upcoming film in the Marvel Cinematic Unive...,23.273,/8chwENebfUEJzZ7sMUA0nOgiCKk.jpg,2027-05-05,Avengers: Secret Wars,False,0.0,0
711875,False,None,"28, 878, 12, 14, 53",216527,en,Avatar 4,,19.103,/qzMYKnT4MG1d0gnhwytr4cKhUvS.jpg,2029-12-19,Avatar 4,False,0.0,0
711876,False,None,"28, 12, 878",393209,en,Avatar 5,,15.253,/rtmmvqkIC5zDMEd638Es2woxbz8.jpg,2031-12-17,Avatar 5,False,0.0,0


In [10]:
# Initialize the genre_names column
df["genre_names"] = df["genre_ids"]

# Loop through the genres dataframe and replace each id with its name
for i in range(len(genres)):
    df["genre_names"] = df["genre_names"].apply(lambda x: x.replace(str(genres["id"][i]), genres["name"][i]) if pd.notnull(x) else x)

In [11]:
# Reducing the dataset to movies with more than 10 votes

df=df[df["vote_count"]>10].reset_index(drop=True)
df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre_names
0,False,/6vYffq6NoneNmNPp6IsobnnlrHS.jpg,99,315946,fr,Passage de Venus,Photo sequence of the rare transit of Venus ov...,5.240,/XWPDZzK7N2WQcejI8W96IxZEeP.jpg,1874-12-09,Passage of Venus,False,6.2,89,Documentary
1,False,None,16,766094,xx,La Rosace Magique,Praxinoscope strip of a shifting rosette. Seri...,0.897,/5zeVMAp3R4QHGarSc4Bf3B1uBY2.jpg,1878-05-07,The Magic Rosette,False,6.1,16,Animation
2,False,None,16,751212,xx,Le Singe Musicien,A pre-cinematograph colour animation of the mo...,0.976,/fx2gnyV2n9KdOCyGzQtVnpB9bIg.jpg,1878-05-07,The Musician Monkey,False,6.3,19,Animation
3,False,None,16,585297,xx,L'Homme Machine,Animated stick drawings representing a man wal...,1.824,/vkxWQys2cqBU7JpWL4PWYG8Va9Z.jpg,1885-01-01,L'Homme Machine,False,4.8,27,Animation
4,False,/gCm6ggiw0YzIZkchHbDWrQbcOiP.jpg,99,159897,xx,Man Walking Around a Corner,The last remaining production of Le Prince's L...,2.890,/3vIpxTUyCFSFagJsGaxFXcR8mOt.jpg,1887-08-18,Man Walking Around a Corner,False,5.0,73,Documentary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70714,False,/msBuBppEFq3QvaRsJYz0r595gC5.jpg,12,1059638,pl,Pan Samochodzik i templariusze,"A well-known art historian, treasure hunter an...",96.109,/xEAXVe0BzW4K9Ky6eTo4CJwzJ8P.jpg,2023-07-12,Mr. Car and the Knights Templar,False,6.4,14,Adventure
70715,False,/g5YC4ek1ZSKogiR5UwiSuTRx4iO.jpg,"16, 35, 10751, 27",1150394,es,La Leyenda de los Chaneques,Five years after giving up his powers and sepa...,7.531,/uybOgGtSW3orlkHTO0qYOQcSgfq.jpg,2023-07-14,La Leyenda de los Chaneques,False,9.1,14,"Animation, Comedy, Family, Horror"
70716,False,/aQ05QNSEersEdOzaqdjnrkqxZXE.jpg,"53, 18, 878, 27",805320,es,Bird Box Barcelona,After a mysterious force decimates the world’s...,228.512,/iTGBHdL12QEpLyRVlykDuiYxzAG.jpg,2023-07-14,Bird Box Barcelona,False,6.0,63,"Thriller, Drama, Science Fiction, Horror"
70717,False,/xcXALwBjdHIjrESpGVhghqj8fGT.jpg,"18, 36, 53",872585,en,Oppenheimer,The story of J. Robert Oppenheimer’s role in t...,324.811,/8Gxv8gSFCU0XGDykEGv7zR1n2ua.jpg,2023-07-19,Oppenheimer,False,8.9,17,"Drama, History, Thriller"


# Cleaning Process

In [12]:
#retrieving only english titles
data=df[df["original_language"]=="en"].reset_index(drop=True)

#drop not needed column
data=data.drop(columns="genre_ids")
data

,adult,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre_names
0,False,None,356155,en,"Glenroy Bros., No. 2",The Glenroy Brothers perform a portion of thei...,0.682,/hlHE0oSgNxQ15AfTlkfESt8XdHO.jpg,1894-01-01,"Glenroy Bros., No. 2",False,4.8,18,Comedy
1,False,None,332353,en,Annabelle Sun Dance,Annabelle Dances in William K.L. Dickson's 189...,0.789,/hLGIzJOzV2Z9kLKZDkMX5yg0dFD.jpg,1894-01-01,Annabelle Sun Dance,False,4.4,14,Documentary
2,False,None,194089,en,The Pickaninny Dance from the “Passing Show”,“A scene representing Southern plantation life...,0.791,None,1894-05-16,The Pickaninny Dance from the “Passing Show”,False,4.0,15,Documentary
3,False,None,159905,en,Leonard-Cushing Fight,"In the background, five fans lean on the ropes...",0.922,None,1894-06-14,Leonard-Cushing Fight,False,5.2,19,Documentary
4,False,None,186021,en,Caicedo (with Pole),King of the slack wire. His daring feats of ba...,1.851,/ccX52HSqICXInX5O5z0DZjLIhTH.jpg,1894-07-25,Caicedo (with Pole),False,5.9,37,Documentary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40590,False,/fjWcAbHRxCSR4kLGvsPEhNjR2ts.jpg,921636,en,The Out-Laws,A straight-laced bank manager is about to marr...,1194.736,/5dliMQ2ODbGNoq0hlefdnuXQxMw.jpg,2023-07-07,The Out-Laws,False,6.3,193,"Romance, Action, Comedy"
40591,False,/628Dep6AxEtDxjZoGP78TsOxYbK.jpg,575264,en,Mission: Impossible - Dead Reckoning Part One,Ethan Hunt and his IMF team embark on their mo...,978.246,/NNxYkU70HPurnNCSiCjYAmacwm.jpg,2023-07-08,Mission: Impossible - Dead Reckoning Part One,False,8.1,274,"Action, Adventure, Thriller"
40592,False,/lv606WMZ9iZpIo6o3yeIVDAEBOl.jpg,1149947,en,To End All War: Oppenheimer & the Atomic Bomb,Explore how one man's relentless drive and inv...,10.543,/xwkYWcs8aypUtk3xDRUh0ndo4GA.jpg,2023-07-09,To End All War: Oppenheimer & the Atomic Bomb,False,9.6,22,"Documentary, History"
40593,False,/xcXALwBjdHIjrESpGVhghqj8fGT.jpg,872585,en,Oppenheimer,The story of J. Robert Oppenheimer’s role in t...,324.811,/8Gxv8gSFCU0XGDykEGv7zR1n2ua.jpg,2023-07-19,Oppenheimer,False,8.9,17,"Drama, History, Thriller"


In [13]:
# 2nd loop to obtain only english titles
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df_english_titles = data[data['original_title'].apply(is_english)]

In [14]:
df_english_titles=df_english_titles.reset_index(drop=True)
df_english_titles

,adult,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre_names
0,False,None,194089,en,The Pickaninny Dance from the “Passing Show”,“A scene representing Southern plantation life...,0.791,None,1894-05-16,The Pickaninny Dance from the “Passing Show”,False,4.0,15,Documentary
1,False,None,159905,en,Leonard-Cushing Fight,"In the background, five fans lean on the ropes...",0.922,None,1894-06-14,Leonard-Cushing Fight,False,5.2,19,Documentary
2,False,None,186021,en,Caicedo (with Pole),King of the slack wire. His daring feats of ba...,1.851,/ccX52HSqICXInX5O5z0DZjLIhTH.jpg,1894-07-25,Caicedo (with Pole),False,5.9,37,Documentary
3,False,/2DWKpmu1qbaIU32Iw6D3tDnlvCw.jpg,104396,en,Dickson Experimental Sound Film,William K.L. Dickson plays the violin while tw...,3.550,/29GejylcrVahrRziYn3B1CLUs9y.jpg,1894-08-31,Dickson Experimental Sound Film,False,6.4,83,Music
4,False,None,194091,en,The Hornbacker-Murphy Fight,A referee and five fans (including a waiter in...,0.963,/jnS7ixhiWCsiXDeBQmemmTgO5lS.jpg,1894-10-02,The Hornbacker-Murphy Fight,False,4.5,20,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26563,False,/eMPxmNvJjxVZIQWI2t1VmNC5IuR.jpg,678512,en,Sound of Freedom,"The story of Tim Ballard, a former US governme...",1745.942,/mx4O9OEvIB265VM3UATLslsSW5t.jpg,2023-07-03,Sound of Freedom,False,8.4,77,"Action, Drama"
26564,False,/hPcP1kv6vrkRmQO3YgV1H97FE5Q.jpg,614479,en,Insidious: The Red Door,"To put their demons to rest once and for all, ...",754.363,/uS1AIL7I1Ycgs8PTfqUeN6jYNsQ.jpg,2023-07-05,Insidious: The Red Door,False,6.0,162,"Horror, Mystery, Thriller"
26565,False,/fjWcAbHRxCSR4kLGvsPEhNjR2ts.jpg,921636,en,The Out-Laws,A straight-laced bank manager is about to marr...,1194.736,/5dliMQ2ODbGNoq0hlefdnuXQxMw.jpg,2023-07-07,The Out-Laws,False,6.3,193,"Romance, Action, Comedy"
26566,False,/628Dep6AxEtDxjZoGP78TsOxYbK.jpg,575264,en,Mission: Impossible - Dead Reckoning Part One,Ethan Hunt and his IMF team embark on their mo...,978.246,/NNxYkU70HPurnNCSiCjYAmacwm.jpg,2023-07-08,Mission: Impossible - Dead Reckoning Part One,False,8.1,274,"Action, Adventure, Thriller"


# Getting budgets, revenues, imdb ids, actors, and streams

In [15]:
df=df_english_titles

#### For search engine

https://github.com/celiao/tmdbsimple/blob/master/README.md

In [16]:
tmdb.API_KEY = 'YOUR_API_KEY' ######## ------------------- select from your TMDB API KEY
tmdb.REQUESTS_SESSION = requests.Session()

In [17]:
### Process takes 1 hour

start_time = time.time()

movie_ids = df["id"].tolist()

movie_data = {}  # Dictionary to store movie data

# Function to fetch movie data for a given movie ID
def fetch_movie_data(movie_id, batch_index):
    try:
        movie = tmdb.Movies(movie_id)
        response = movie.info()
        budget = response.get("budget")
        revenue = response.get("revenue")
        imdb_id = response.get("imdb_id")

        response2 = movie.credits()
        actor_names = ', '.join([actor['name'] for actor in response2['cast']])

        response3 = movie.watch_providers()
        watch_providers = response3.get("results", {})
        provider_names = ""
        if watch_providers:
            us_providers = watch_providers.get("US", {})
            if us_providers:
                provider_names = ', '.join([provider['provider_name'] for provider in us_providers.get("flatrate", [])])
        if not provider_names:
            provider_names = "Not found"

        # Store the data in the dictionary with movie ID as key
        movie_data[movie_id] = {
            "budget": budget,
            "revenue": revenue,
            "imdb_id": imdb_id,
            "actors": actor_names,
            "providers": provider_names
        }

        #print(f"Batch {batch_index}: Processed movie ID {movie_id}")

    except requests.exceptions.RequestException:
        print(f"Batch {batch_index}: Error occurred for movie ID {movie_id}")

# Function to process a batch of movie IDs using multi-threading
def process_batch(batch_ids, batch_index):
    threads = []
    for movie_id in batch_ids:
        thread = threading.Thread(target=fetch_movie_data, args=(movie_id, batch_index))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

# Set the batch size for concurrent processing
batch_size = 20

# Process movie IDs in batches
for i in range(0, len(movie_ids), batch_size):
    batch_ids = movie_ids[i:i+batch_size]
    process_batch(batch_ids, i // batch_size)

# Add the retrieved data to the DataFrame
df["budget"] = df["id"].map(lambda x: movie_data.get(x, {}).get("budget"))
df["revenue"] = df["id"].map(lambda x: movie_data.get(x, {}).get("revenue"))
df["imdb_id"] = df["id"].map(lambda x: movie_data.get(x, {}).get("imdb_id"))
df['actors'] = df["id"].map(lambda x: movie_data.get(x, {}).get("actors"))
df['providers'] = df["id"].map(lambda x: movie_data.get(x, {}).get("providers"))

total_time = time.time() - start_time
print(f"Total execution time: {total_time} seconds")

df

Batch 0: Processed movie ID 146576
Batch 0: Processed movie ID 104396
Batch 0: Processed movie ID 186021
Batch 0: Processed movie ID 127221
Batch 0: Processed movie ID 178696
Batch 0: Processed movie ID 194179
Batch 0: Processed movie ID 194089
Batch 0: Processed movie ID 178995
Batch 0: Processed movie ID 355763
Batch 0: Processed movie ID 146579
Batch 0: Processed movie ID 426904
Batch 0: Processed movie ID 159905
Batch 0: Processed movie ID 146565
Batch 0: Processed movie ID 178998
Batch 0: Processed movie ID 189270
Batch 0: Processed movie ID 194091
Batch 0: Processed movie ID 208463
Batch 0: Processed movie ID 146580
Batch 0: Processed movie ID 146574
Batch 0: Processed movie ID 146573
Batch 1: Processed movie ID 147610
Batch 1: Processed movie ID 152207
Batch 1: Processed movie ID 105166
Batch 1: Processed movie ID 178996
Batch 1: Processed movie ID 568034
Batch 1: Processed movie ID 152219
Batch 1: Processed movie ID 178700
Batch 1: Processed movie ID 146522
Batch 1: Processed m

Batch 12: Processed movie ID 30982
Batch 12: Processed movie ID 618
Batch 12: Processed movie ID 70368
Batch 12: Processed movie ID 2981
Batch 12: Processed movie ID 53405
Batch 12: Processed movie ID 43678
Batch 12: Processed movie ID 53402
Batch 12: Processed movie ID 53406
Batch 12: Processed movie ID 53412
Batch 12: Processed movie ID 53408
Batch 12: Processed movie ID 151085
Batch 12: Processed movie ID 150538
Batch 12: Processed movie ID 53403
Batch 12: Processed movie ID 86081
Batch 12: Processed movie ID 53409
Batch 12: Processed movie ID 50775
Batch 12: Processed movie ID 227061
Batch 12: Processed movie ID 52276
Batch 12: Processed movie ID 208517
Batch 12: Processed movie ID 53410
Batch 13: Processed movie ID 31510
Batch 13: Processed movie ID 32600
Batch 13: Processed movie ID 3060
Batch 13: Processed movie ID 964
Batch 13: Processed movie ID 98477
Batch 13: Processed movie ID 121379
Batch 13: Processed movie ID 42359
Batch 13: Processed movie ID 106061
Batch 13: Processed 

Batch 23: Processed movie ID 117531
Batch 23: Processed movie ID 47571
Batch 23: Processed movie ID 191381
Batch 23: Processed movie ID 67175
Batch 23: Processed movie ID 70879
Batch 24: Processed movie ID 64398
Batch 24: Processed movie ID 2760
Batch 24: Processed movie ID 27503
Batch 24: Processed movie ID 77022
Batch 24: Processed movie ID 50897
Batch 24: Processed movie ID 85638
Batch 24: Processed movie ID 61546
Batch 24: Processed movie ID 16661
Batch 24: Processed movie ID 170979
Batch 24: Processed movie ID 68408
Batch 24: Processed movie ID 50920
Batch 24: Processed movie ID 47695
Batch 24: Processed movie ID 50930
Batch 24: Processed movie ID 144615
Batch 24: Processed movie ID 41111
Batch 24: Processed movie ID 82283
Batch 24: Processed movie ID 49633
Batch 24: Processed movie ID 120829
Batch 24: Processed movie ID 106555
Batch 24: Processed movie ID 44657
Batch 25: Processed movie ID 82474
Batch 25: Processed movie ID 28978
Batch 25: Processed movie ID 631
Batch 25: Process

Batch 35: Processed movie ID 75510
Batch 35: Processed movie ID 81107
Batch 35: Processed movie ID 43131
Batch 35: Processed movie ID 56724
Batch 35: Processed movie ID 26860
Batch 35: Processed movie ID 76964
Batch 35: Processed movie ID 156462
Batch 35: Processed movie ID 156500
Batch 35: Processed movie ID 26714
Batch 35: Processed movie ID 156454
Batch 35: Processed movie ID 77966
Batch 35: Processed movie ID 161581
Batch 36: Processed movie ID 1994
Batch 36: Processed movie ID 31592
Batch 36: Processed movie ID 31532
Batch 36: Processed movie ID 44775
Batch 36: Processed movie ID 13912
Batch 36: Processed movie ID 38077
Batch 36: Processed movie ID 43075
Batch 36: Processed movie ID 13634
Batch 36: Processed movie ID 73125
Batch 36: Processed movie ID 84303
Batch 36: Processed movie ID 1773
Batch 36: Processed movie ID 156464
Batch 36: Processed movie ID 80193
Batch 36: Processed movie ID 82970
Batch 36: Processed movie ID 156461
Batch 36: Processed movie ID 23253
Batch 36: Proces

Batch 47: Processed movie ID 150316
Batch 47: Processed movie ID 39441
Batch 47: Processed movie ID 148075
Batch 47: Processed movie ID 49011
Batch 47: Processed movie ID 41073
Batch 47: Processed movie ID 87237
Batch 47: Processed movie ID 29241
Batch 48: Processed movie ID 40164
Batch 48: Processed movie ID 18771
Batch 48: Processed movie ID 32430
Batch 48: Processed movie ID 851
Batch 48: Processed movie ID 68492
Batch 48: Processed movie ID 17645
Batch 48: Processed movie ID 17058
Batch 48: Processed movie ID 43488
Batch 48: Processed movie ID 31561
Batch 48: Processed movie ID 30793
Batch 48: Processed movie ID 22612
Batch 48: Processed movie ID 61392
Batch 48: Processed movie ID 76068
Batch 48: Processed movie ID 28580
Batch 48: Processed movie ID 64077
Batch 48: Processed movie ID 179254
Batch 48: Processed movie ID 15401
Batch 48: Processed movie ID 48890
Batch 48: Processed movie ID 67585
Batch 48: Processed movie ID 132392
Batch 49: Processed movie ID 81077
Batch 49: Processe

...

Batch 1289: Processed movie ID 825808
Batch 1289: Processed movie ID 910772
Batch 1289: Processed movie ID 914268
Batch 1289: Processed movie ID 887309
Batch 1289: Processed movie ID 926899
Batch 1289: Processed movie ID 860135
Batch 1289: Processed movie ID 874136
Batch 1289: Processed movie ID 890233
Batch 1290: Processed movie ID 245842
Batch 1290: Processed movie ID 697799
Batch 1290: Processed movie ID 892153
Batch 1290: Processed movie ID 656663
Batch 1290: Processed movie ID 891931
Batch 1290: Processed movie ID 803114
Batch 1290: Processed movie ID 774825
Batch 1290: Processed movie ID 817648
Batch 1290: Processed movie ID 689700
Batch 1290: Processed movie ID 928773
Batch 1290: Processed movie ID 919961
Batch 1290: Processed movie ID 928015
Batch 1290: Processed movie ID 870671
Batch 1290: Processed movie ID 931871
Batch 1290: Processed movie ID 736280
Batch 1290: Processed movie ID 929326
Batch 1290: Processed movie ID 929562
Batch 1290: Processed movie ID 947612
Batch 1290: 

Batch 1300: Processed movie ID 846250
Batch 1300: Processed movie ID 899294
Batch 1300: Processed movie ID 984430
Batch 1300: Processed movie ID 507086
Batch 1300: Processed movie ID 965398
Batch 1300: Processed movie ID 913868
Batch 1300: Processed movie ID 980856
Batch 1300: Processed movie ID 943776
Batch 1300: Processed movie ID 829577
Batch 1300: Processed movie ID 942576
Batch 1300: Processed movie ID 931346
Batch 1301: Processed movie ID 913814
Batch 1301: Processed movie ID 834742
Batch 1301: Processed movie ID 718789
Batch 1301: Processed movie ID 916735
Batch 1301: Processed movie ID 952217
Batch 1301: Processed movie ID 759175
Batch 1301: Processed movie ID 668047
Batch 1301: Processed movie ID 975714
Batch 1301: Processed movie ID 814340
Batch 1301: Processed movie ID 972313
Batch 1301: Processed movie ID 758330
Batch 1301: Processed movie ID 893086
Batch 1301: Processed movie ID 985075
Batch 1301: Processed movie ID 967183
Batch 1301: Processed movie ID 762006
Batch 1301: 

Batch 1311: Processed movie ID 1013572
Batch 1311: Processed movie ID 914216
Batch 1311: Processed movie ID 1028703
Batch 1311: Processed movie ID 977367
Batch 1311: Processed movie ID 1061089
Batch 1311: Processed movie ID 1033024
Batch 1311: Processed movie ID 881487
Batch 1311: Processed movie ID 772973
Batch 1311: Processed movie ID 860854
Batch 1312: Processed movie ID 1026202
Batch 1312: Processed movie ID 992277
Batch 1312: Processed movie ID 852252
Batch 1312: Processed movie ID 603204
Batch 1312: Processed movie ID 654910
Batch 1312: Processed movie ID 843847
Batch 1312: Processed movie ID 1059241
Batch 1312: Processed movie ID 1035406
Batch 1312: Processed movie ID 707103
Batch 1312: Processed movie ID 879444
Batch 1312: Processed movie ID 869025
Batch 1312: Processed movie ID 1040330
Batch 1312: Processed movie ID 541134
Batch 1312: Processed movie ID 864959
Batch 1312: Processed movie ID 888479
Batch 1312: Processed movie ID 1019105
Batch 1312: Processed movie ID 766220
Bat

Batch 1322: Processed movie ID 852096
Batch 1322: Processed movie ID 1081789
Batch 1322: Processed movie ID 845659
Batch 1322: Processed movie ID 844417
Batch 1322: Processed movie ID 869112
Batch 1322: Processed movie ID 1063422
Batch 1322: Processed movie ID 1090241
Batch 1322: Processed movie ID 1075334
Batch 1322: Processed movie ID 804150
Batch 1322: Processed movie ID 971188
Batch 1322: Processed movie ID 1078764
Batch 1322: Processed movie ID 722149
Batch 1322: Processed movie ID 677179
Batch 1322: Processed movie ID 785759
Batch 1322: Processed movie ID 703451
Batch 1323: Processed movie ID 1084072
Batch 1323: Processed movie ID 943788
Batch 1323: Processed movie ID 953365
Batch 1323: Processed movie ID 1081683
Batch 1323: Processed movie ID 872516
Batch 1323: Processed movie ID 714964
Batch 1323: Processed movie ID 1036311
Batch 1323: Processed movie ID 1087040
Batch 1323: Processed movie ID 1100287
Batch 1323: Processed movie ID 933419
Batch 1323: Processed movie ID 594767
Ba

,adult,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre_names,budget,revenue,imdb_id,actors,providers
0,False,None,194089,en,The Pickaninny Dance from the “Passing Show”,“A scene representing Southern plantation life...,0.791,None,1894-05-16,The Pickaninny Dance from the “Passing Show”,False,4.0,15,Documentary,0,0,tt0285903,"Joe Rastus, Denny Tolliver, Walter Wilkins",Not found
1,False,None,159905,en,Leonard-Cushing Fight,"In the background, five fans lean on the ropes...",0.922,None,1894-06-14,Leonard-Cushing Fight,False,5.2,19,Documentary,0,0,tt0203646,"Jack Cushing, Mike Leonard",Not found
2,False,None,186021,en,Caicedo (with Pole),King of the slack wire. His daring feats of ba...,1.851,/ccX52HSqICXInX5O5z0DZjLIhTH.jpg,1894-07-25,Caicedo (with Pole),False,5.9,37,Documentary,0,0,tt0241282,Juan A. Caicedo,Not found
3,False,/2DWKpmu1qbaIU32Iw6D3tDnlvCw.jpg,104396,en,Dickson Experimental Sound Film,William K.L. Dickson plays the violin while tw...,3.550,/29GejylcrVahrRziYn3B1CLUs9y.jpg,1894-08-31,Dickson Experimental Sound Film,False,6.4,83,Music,0,0,tt0177707,"William K.L. Dickson, Charles D'Almaine",Not found
4,False,None,194091,en,The Hornbacker-Murphy Fight,A referee and five fans (including a waiter in...,0.963,/jnS7ixhiWCsiXDeBQmemmTgO5lS.jpg,1894-10-02,The Hornbacker-Murphy Fight,False,4.5,20,,0,0,tt0477315,Eugene Hornbacker,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26563,False,/eMPxmNvJjxVZIQWI2t1VmNC5IuR.jpg,678512,en,Sound of Freedom,"The story of Tim Ballard, a former US governme...",1745.942,/mx4O9OEvIB265VM3UATLslsSW5t.jpg,2023-07-03,Sound of Freedom,False,8.4,77,"Action, Drama",0,40000000,tt7599146,"Jim Caviezel, Mira Sorvino, Bill Camp, Kurt Fu...",Not found
26564,False,/hPcP1kv6vrkRmQO3YgV1H97FE5Q.jpg,614479,en,Insidious: The Red Door,"To put their demons to rest once and for all, ...",754.363,/uS1AIL7I1Ycgs8PTfqUeN6jYNsQ.jpg,2023-07-05,Insidious: The Red Door,False,6.0,162,"Horror, Mystery, Thriller",16000000,64000000,tt13405778,"Ty Simpkins, Patrick Wilson, Rose Byrne, Lin S...",Not found
26565,False,/fjWcAbHRxCSR4kLGvsPEhNjR2ts.jpg,921636,en,The Out-Laws,A straight-laced bank manager is about to marr...,1194.736,/5dliMQ2ODbGNoq0hlefdnuXQxMw.jpg,2023-07-07,The Out-Laws,False,6.3,193,"Romance, Action, Comedy",0,0,tt11274492,"Pierce Brosnan, Adam DeVine, Nina Dobrev, Elle...","Netflix, Netflix basic with Ads"
26566,False,/628Dep6AxEtDxjZoGP78TsOxYbK.jpg,575264,en,Mission: Impossible - Dead Reckoning Part One,Ethan Hunt and his IMF team embark on their mo...,978.246,/NNxYkU70HPurnNCSiCjYAmacwm.jpg,2023-07-08,Mission: Impossible - Dead Reckoning Part One,False,8.1,274,"Action, Adventure, Thriller",290000000,0,tt9603212,"Tom Cruise, Hayley Atwell, Ving Rhames, Simon ...",Not found


In [18]:
df.to_csv("TMDB_movies_final.csv",index=False)

-------